In [16]:
import os

# Vérifier que le dossier data/ existe
data_folder = 'Data'

print("Fichiers dans le dossier 'Data/' :")
print("="*50)

if os.path.exists(data_folder):
    files = os.listdir(data_folder)
    for i, file in enumerate(files, 1):
        print(f"{i}. {file}")
else:
    print("Le dossier 'Data/' n'existe pas !")

Fichiers dans le dossier 'Data/' :
1. .ipynb_checkpoints
2. auto-mpg.csv
3. BreastCanDT.csv
4. concrete_data.csv
5. dataset_scenario1.csv
6. dataset_scenario2.csv
7. dataset_scenario3.csv
8. dataset_scenario4.csv
9. HousingData.csv
10. ozone.csv
11. parkinsons.csv
12. ReplicatedAcousticFeatures-ParkinsonDatabase.csv
13. RLT_PROJECT.ipynb
14. sonar.csv
15. winequality-red.csv
16. winequality-white.csv


In [17]:
"""
============================================================================
AUGMENTATION DES FEATURES À 500
============================================================================
Ce script charge chaque dataset et augmente le nombre de features à 500
en créant des features synthétiques (signal + bruit, ratio 1:2)
============================================================================
"""

import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

# ============================================================================
# CONFIGURATION
# ============================================================================

TARGET_P = 500
SEED = 42
SIGNAL_NOISE_RATIO = 0.33
OUTPUT_FOLDER = 'datasets_augmented'

DATASETS_CONFIG = [
    {'name': 'HousingData', 'filepath': 'Data/HousingData.csv', 'target_col': 'MEDV', 'sep': ','},
    {'name': 'BreastCanDT', 'filepath': 'Data/BreastCanDT.csv', 'target_col': 'diagnosis', 'sep': ','},
    {'name': 'parkinsons', 'filepath': 'data/parkinsons.csv', 'target_col': 'status', 'sep': ','},
    {'name': 'sonar', 'filepath': 'data/sonar.csv', 'target_col': 'R', 'sep': ','},
    {'name': 'winequality-white', 'filepath': 'Data/winequality-white.csv', 'target_col': 'quality', 'sep': ';'},
    {'name': 'winequality-red', 'filepath': 'Data/winequality-red.csv', 'target_col': 'quality', 'sep': ','},
    {'name': 'ReplicatedAcousticFeatures-ParkinsonDatabase',
     'filepath': 'Data/ReplicatedAcousticFeatures-ParkinsonDatabase.csv',
     'target_col': 'Status', 'sep': ','},
    {'name': 'ozone', 'filepath': 'Data/ozone.csv', 'target_col': 'maxO3', 'sep': ','},
    {'name': 'concrete_data', 'filepath': 'Data/concrete_data.csv',
     'target_col': 'concrete_compressive_strength', 'sep': ','},
    {'name': 'auto_mpg', 'filepath': 'Data/auto-mpg.csv', 'target_col': 'mpg', 'sep': ','}
]

# ============================================================================
# FEATURE AUGMENTATION
# ============================================================================

def augment_features(X, target_p=TARGET_P, seed=SEED):
    np.random.seed(seed)

    n_samples, p_original = X.shape
    n_new = target_p - p_original

    if n_new <= 0:
        print(f"       Déjà {p_original} features")
        return X

    print(f"      Augmentation : {p_original} → {target_p} features")

    X_new = np.zeros((n_samples, n_new))

    for i in range(n_new):
        idx = np.random.randint(0, p_original)
        signal = X[:, idx]

        std = np.std(signal)
        noise = np.random.normal(0, std if std > 0 else 1.0, n_samples)

        X_new[:, i] = SIGNAL_NOISE_RATIO * signal + (1 - SIGNAL_NOISE_RATIO) * noise

    return np.hstack([X, X_new])

# ============================================================================
# DATASET PROCESSING
# ============================================================================

def process_dataset(config):
    print(f"\n{'='*70}")
    print(f" {config['name']}")
    print(f"{'='*70}")

    try:
        df = pd.read_csv(config['filepath'], sep=config['sep'])
        print(f"   Shape originale : {df.shape}")

        target_col = config['target_col']
        y = df[target_col]
        X_df = df.drop(columns=[target_col])

        # garder seulement les features numériques
        X_df = X_df.select_dtypes(include=[np.number])
        original_column_names = X_df.columns.tolist()

        # encoder la target si nécessaire
        mapping = None
        if y.dtype == 'object':
            le = LabelEncoder()
            y = le.fit_transform(y)
            mapping = {int(i): str(v) for i, v in enumerate(le.classes_)}
            print(f"   Target encode : {mapping}")
        else:
            y = y.values

        # =======================
        # IMPUTATION DES NaN
        # =======================
        if X_df.isna().any().any():
            print("   Valeurs manquantes détectées")

            for col_idx in range(X_df.shape[1]):
                col = X_df.iloc[:, col_idx]

                if not col.isna().any():
                    continue

                unique_vals = col.dropna().unique()

                # colonne entièrement NaN
                if len(unique_vals) == 0:
                    X_df.iloc[:, col_idx] = 0.0
                    print(f"      Colonne {col_idx} : entièrement NaN → 0")
                    continue

                is_binary = set(unique_vals).issubset({0, 1, 0.0, 1.0})

                if is_binary:
                    mode_val = col.mode()[0]
                    X_df.iloc[:, col_idx] = col.fillna(mode_val)
                    print(f"      Colonne {col_idx} (binaire) : mode = {mode_val}")
                else:
                    median_val = col.median()
                    X_df.iloc[:, col_idx] = col.fillna(median_val)
                    print(f"      Colonne {col_idx} (numérique) : médiane = {median_val:.2f}")

        X = X_df.values.astype(float)
        y = y.astype(float)

        print(f"   X : {X.shape}, y : {y.shape}")

        # augmentation
        X_augmented = augment_features(X)

        os.makedirs(OUTPUT_FOLDER, exist_ok=True)

        n_original = len(original_column_names)
        n_synthetic = X_augmented.shape[1] - n_original

        columns = original_column_names + [f"synthetic_{i}" for i in range(n_synthetic)]
        output_df = pd.DataFrame(X_augmented, columns=columns)
        output_df[target_col] = y

        output_path = os.path.join(OUTPUT_FOLDER, f"{config['name']}.csv")
        output_df.to_csv(output_path, index=False)

        print(f"    Sauvegardé : {output_path}")

        return {'name': config['name'], 'status': 'success',
                'shape': X_augmented.shape, 'mapping': mapping}

    except Exception as e:
        print(f"    ERREUR : {e}")
        return {'name': config['name'], 'status': 'failed', 'error': str(e)}

# ============================================================================
# MAIN
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("AUGMENTATION DES FEATURES À 500")
    print("="*70)

    results = [process_dataset(cfg) for cfg in DATASETS_CONFIG]

    print("\n" + "="*70)
    print("RÉSUMÉ")
    print("="*70)

    success = [r for r in results if r['status'] == 'success']
    failed = [r for r in results if r['status'] == 'failed']

    print(f"\n Réussis : {len(success)}/{len(results)}")
    for r in success:
        print(f"   {r['name']:<50s} → {r['shape']}")

    if failed:
        print(f"\n Échoués : {len(failed)}")
        for r in failed:
            print(f"   {r['name']} : {r['error']}")

    print(f"\n Datasets sauvegardés dans : {OUTPUT_FOLDER}/")
    print("\n TERMINÉ !")


AUGMENTATION DES FEATURES À 500

 HousingData
   Shape originale : (506, 14)
   Valeurs manquantes détectées
      Colonne 0 (numérique) : médiane = 0.25
      Colonne 1 (numérique) : médiane = 0.00
      Colonne 2 (numérique) : médiane = 9.69
      Colonne 3 (binaire) : mode = 0.0
      Colonne 6 (numérique) : médiane = 76.80
      Colonne 12 (numérique) : médiane = 11.43
   X : (506, 13), y : (506,)
      Augmentation : 13 → 500 features
    Sauvegardé : datasets_augmented\HousingData.csv

 BreastCanDT
   Shape originale : (569, 33)
   Target encode : {0: 'B', 1: 'M'}
   Valeurs manquantes détectées
      Colonne 31 : entièrement NaN → 0
   X : (569, 32), y : (569,)
      Augmentation : 32 → 500 features
    Sauvegardé : datasets_augmented\BreastCanDT.csv

 parkinsons
   Shape originale : (195, 24)
   X : (195, 22), y : (195,)
      Augmentation : 22 → 500 features
    Sauvegardé : datasets_augmented\parkinsons.csv

 sonar
   Shape originale : (207, 61)
   Target encode : {0: 'M', 1: